### Exploratory Data Analysis of Ridesharing Dataset

#### Overview:
This Jupyter notebook contains code and analysis for exploring the New York Ridesharing Dataset. The dataset comprises Ride Sharing Data, including time, longitute, latitude, and other relevant details.

#### Dataset Used:
- Dataset Name: New York City Ridesharing Dataset
- Source: https://www.kaggle.com/datasets/fivethirtyeight/uber-pickups-in-new-york-city

#### Objective:
The primary goal of this notebook is to perform exploratory data analysis (EDA) on the Uber Ridesharing Dataset. This includes:
- Data cleaning and preprocessing
- Merging data for training datasets
- Extracting insights and patterns from the data
- Rudementary visualizations for quick analysis

### Author:
- Name: Aden Letchworth
- Date: 12/17/2023




In [1]:
# Import Libraries

import pandas as pd
import plotly.express as px

import sys

# Custom Helper Functions

sys.path.append('../src')
import data_utils as ds 

### Basic Data Exploration

**Goals** 
1. Load CSV
2. Understand Data (Cols, Dtypes, Format, Etc)
3. Verify Integrity
4. Find any issues with formatting then move to cleaning and visualization

In [2]:
# Load Dataset (1)

df = pd.read_csv('../data/raw/uber-raw-data-sep14.csv')

In [3]:
# Understand formatting of dataset (2)

df.head

<bound method NDFrame.head of                   Date/Time      Lat      Lon    Base
0          9/1/2014 0:01:00  40.2201 -74.0021  B02512
1          9/1/2014 0:01:00  40.7500 -74.0027  B02512
2          9/1/2014 0:03:00  40.7559 -73.9864  B02512
3          9/1/2014 0:06:00  40.7450 -73.9889  B02512
4          9/1/2014 0:11:00  40.8145 -73.9444  B02512
...                     ...      ...      ...     ...
1028131  9/30/2014 22:57:00  40.7668 -73.9845  B02764
1028132  9/30/2014 22:57:00  40.6911 -74.1773  B02764
1028133  9/30/2014 22:58:00  40.8519 -73.9319  B02764
1028134  9/30/2014 22:58:00  40.7081 -74.0066  B02764
1028135  9/30/2014 22:58:00  40.7140 -73.9496  B02764

[1028136 rows x 4 columns]>

In [4]:
# Understand data types of dataset (2)

print(df.columns)
print(df.dtypes)

Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')
Date/Time     object
Lat          float64
Lon          float64
Base          object
dtype: object


In [5]:
# Verify Integrity of Dataset (3)

print(f'Null Values: {df.isnull().any().any()}, NA Values: {df.isna().any().any()}')

Null Values: False, NA Values: False


In [6]:
# Check Date formatting, aggregation shows any potential problems with visualization (4)

df['Date/Time'].value_counts

<bound method IndexOpsMixin.value_counts of 0            9/1/2014 0:01:00
1            9/1/2014 0:01:00
2            9/1/2014 0:03:00
3            9/1/2014 0:06:00
4            9/1/2014 0:11:00
                  ...        
1028131    9/30/2014 22:57:00
1028132    9/30/2014 22:57:00
1028133    9/30/2014 22:58:00
1028134    9/30/2014 22:58:00
1028135    9/30/2014 22:58:00
Name: Date/Time, Length: 1028136, dtype: object>

We can see here the Date/Time data is in an odd format.
- If we want to aggregate properly we must ignore parameters (time, day, month)
- Date format needs leading zeros

We can see the aggregations we can make for visualizations. 
- Aggregate by time of day
- Aggregate by day of week
- Aggregate by month of year

-----

### Data Cleaning & with Basic Visualizations

**Goals**
1. Create filter functions for desired aggregations
2. Apply filter functiosn for custom columns
3. Make visualizations to aid in verifying filters worked and understanding data further
4. Merge uber ridesharing datasets (7 total)

#### Create Regex function for Isolating Date 

Notice: Taking Substrings wouldn't work correctly since the date is formatted '9/30/2014...' in some cases so we would have to take it in accordance to string length, however the suffix can be different sizes such as '0:01:00' and '22:57:00' making regex the simplest way of extracting the date.

In [7]:
"""
HERE FOR DOCCUMENTATION PURPOSES MOVED TO DATA UTILITY FILE

def find_date(text):
    date_pattern = r'^(\d{1,2}/\d{1,2}/\d{4})'  
    match = re.search(date_pattern, text)
    if match:
        return match.group(1)  
    else:
        return None 

def find_time(text):
    time_pattern = r'^(\d{1,2}:\d{2}:\d{2})'  
    match = re.search(time_pattern, text)
    if match:
        return match.group(1)  
    else:
        return None

def standardize_date(text):
    date_pattern_month = r'^(\d{1}/\d{2}/\d{4})'  
    date_pattern_day = r'^(\d{2}/\d{1}/\d{4})'    
    date_pattern_all = r'^(\d{1}/\d{1}/\d{4})'   

    if re.match(date_pattern_month, text):
        return re.sub(r'^(\d{1}/)(\d{2}/\d{4})', r'0\1\2', text)
    
    if re.match(date_pattern_day, text):
        return re.sub(r'^(\d{2}/)(\d{1}/\d{4})', r'\10\2', text)

    if re.match(date_pattern_all, text):
        return re.sub(r'^(\d{1}/)(\d{1}/\d{4})', r'0\1\2', text)

    return text  

"""

"\nHERE FOR DOCCUMENTATION PURPOSES MOVED TO DATA UTILITY FILE\n\ndef find_date(text):\n    date_pattern = r'^(\\d{1,2}/\\d{1,2}/\\d{4})'  \n    match = re.search(date_pattern, text)\n    if match:\n        return match.group(1)  \n    else:\n        return None \n\ndef find_time(text):\n    time_pattern = r'^(\\d{1,2}:\\d{2}:\\d{2})'  \n    match = re.search(time_pattern, text)\n    if match:\n        return match.group(1)  \n    else:\n        return None\n\ndef standardize_date(text):\n    date_pattern_month = r'^(\\d{1}/\\d{2}/\\d{4})'  \n    date_pattern_day = r'^(\\d{2}/\\d{1}/\\d{4})'    \n    date_pattern_all = r'^(\\d{1}/\\d{1}/\\d{4})'   \n\n    if re.match(date_pattern_month, text):\n        return re.sub(r'^(\\d{1}/)(\\d{2}/\\d{4})', r'0\x01\x02', text)\n    \n    if re.match(date_pattern_day, text):\n        return re.sub(r'^(\\d{2}/)(\\d{1}/\\d{4})', r'\x08\x02', text)\n\n    if re.match(date_pattern_all, text):\n        return re.sub(r'^(\\d{1}/)(\\d{1}/\\d{4})', r'0\x01

In [8]:
# Apply date and time functions to dataset (2)

df['Date'] = df['Date/Time'].apply(lambda x: ds.find_date(x))

df['Time'] = df['Date/Time'].apply(lambda x: ds.find_time(x))

df['Date'] = df['Date'].apply(lambda x: ds.standardize_date(x))

In [9]:
df

,Date/Time,Lat,Lon,Base,Date,Time
0,9/1/2014 0:01:00,40.2201,-74.0021,B02512,09/01/2014,None
1,9/1/2014 0:01:00,40.7500,-74.0027,B02512,09/01/2014,None
2,9/1/2014 0:03:00,40.7559,-73.9864,B02512,09/01/2014,None
3,9/1/2014 0:06:00,40.7450,-73.9889,B02512,09/01/2014,None
4,9/1/2014 0:11:00,40.8145,-73.9444,B02512,09/01/2014,None
...,...,...,...,...,...,...
1028131,9/30/2014 22:57:00,40.7668,-73.9845,B02764,09/30/2014,None
1028132,9/30/2014 22:57:00,40.6911,-74.1773,B02764,09/30/2014,None
1028133,9/30/2014 22:58:00,40.8519,-73.9319,B02764,09/30/2014,None
1028134,9/30/2014 22:58:00,40.7081,-74.0066,B02764,09/30/2014,None


In [10]:
# Perform frequency count of dates
date_counts = df['Date'].value_counts().reset_index()
date_counts.columns = ['Date', 'Frequency']

# Sort the date_counts DataFrame by 'Date' column in ascending order
date_counts = date_counts.sort_values('Date')

# Create Plotly bar chart for date frequency distribution
fig = px.bar(date_counts, x='Date', y='Frequency', labels={'Date': 'Date', 'Frequency': 'Frequency Count'})
fig.update_xaxes(type='category')  
fig.update_layout(title='Date Frequency Distribution (Sorted by Date)')
fig.show()

In [11]:
# Perform frequency count of times
time_counts = df['Time'].value_counts().reset_index()
time_counts.columns = ['Time', 'Frequency']

# Sort the time_counts DataFrame by 'Time' column in ascending order
time_counts = time_counts.sort_values('Time')

# Create Plotly bar chart for time frequency distribution
fig = px.bar(time_counts, x='Time', y='Frequency', labels={'Time': 'Time', 'Frequency': 'Frequency Count'})
fig.update_xaxes(type='category')
fig.update_layout(title='Time Frequency Distribution (Sorted by Time)')
fig.show()

In [12]:
# Get day of week from date

df['Day'] = df['Date'].apply(lambda x: ds.get_day_of_week_from_string(x))

In [13]:
# Perform frequency count of days
days_counts = df['Day'].value_counts().reset_index()
days_counts.columns = ['Day', 'Frequency']

# Create Plotly bar chart for day frequency distribution
fig = px.bar(days_counts, x='Day', y='Frequency', labels={'Day': 'Day', 'Frequency': 'Frequency Count'})
fig.update_xaxes(type='category')  
fig.update_layout(title='Day Frequency Distribution (Sorted by Frequency)')
fig.show()

In [14]:
files = ds.get_files_by_regex('../data/raw/uber-raw-data*.csv')

data_frames = []

for file in files:
    data_frames.append(pd.read_csv(file))

for data_frame in data_frames:
    print(data_frame.columns)

Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')
Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')
Index(['Dispatching_base_num', 'Pickup_date', 'Affiliated_base_num',
       'locationID'],
      dtype='object')
Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')
Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')
Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')
Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')


In [15]:
data_frames[2]

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID
0,B02617,2015-05-17 09:47:00,B02617,141
1,B02617,2015-05-17 09:47:00,B02617,65
2,B02617,2015-05-17 09:47:00,B02617,100
3,B02617,2015-05-17 09:47:00,B02774,80
4,B02617,2015-05-17 09:47:00,B02617,90
...,...,...,...,...
14270474,B02765,2015-05-08 15:43:00,B02765,186
14270475,B02765,2015-05-08 15:43:00,B02765,263
14270476,B02765,2015-05-08 15:43:00,B02765,90
14270477,B02765,2015-05-08 15:44:00,B01899,45


In [16]:
del data_frames[2]

In [17]:
for data_frame in data_frames:
    print(data_frame.columns)

Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')
Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')
Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')
Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')
Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')
Index(['Date/Time', 'Lat', 'Lon', 'Base'], dtype='object')


In [18]:
from functools import reduce

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date/Time', 'Lat', 'Lon', 'Base'], how='outer'), data_frames)

df_merged

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512
...,...,...,...,...
4534322,9/30/2014 22:57:00,40.7668,-73.9845,B02764
4534323,9/30/2014 22:57:00,40.6911,-74.1773,B02764
4534324,9/30/2014 22:58:00,40.8519,-73.9319,B02764
4534325,9/30/2014 22:58:00,40.7081,-74.0066,B02764


In [19]:
df_merged.to_csv('../data/processed/uber-raw-data-merged.csv', index=False)

In [20]:
print(f'Null Values: {df_merged.isnull().any().any()}, NA Values: {df_merged.isna().any().any()}')

Null Values: False, NA Values: False


In [21]:
df_merged['Date'] = df_merged['Date/Time'].apply(lambda x: ds.find_date(x))

df_merged['Date'] = df_merged['Date'].apply(lambda x: ds.standardize_date(x))

KeyboardInterrupt: 

In [ ]:
df_merged['Day'] = df_merged['Date'].apply(lambda x: ds.get_day_of_week_from_string(x))

In [ ]:
# Perform frequency count of days
days_counts = df_merged['Day'].value_counts().reset_index()
days_counts.columns = ['Day', 'Frequency']

# Create Plotly bar chart for day frequency distribution
fig = px.bar(days_counts, x='Day', y='Frequency', labels={'Day': 'Day', 'Frequency': 'Frequency Count'})
fig.update_xaxes(type='category')  
fig.update_layout(title='Day Frequency Distribution (Sorted by Frequency)')
fig.show()

In [ ]:
df_merged.to_csv('../data/processed/uber-data-merged.csv', index=False)

In [ ]:
clean_df = pd.read_csv('../data/processed/uber-data-merged.csv')